In [1]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 39 
num_classes = 7

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [2]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [3]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017Kharif_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/Gamma10m_2017Kharif_Gamma_VV1.csv', delimiter=',')
my_data0 = genfromtxt('./DataForBiplab/Gamma10m_2017Kharif_Gamma_VH1.csv', delimiter=',')

In [4]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
data0 = my_data0[1:,2:]
print(data.shape)
print(data1.shape)
print(data0.shape)

(546, 13)
(546, 13)
(546, 13)


In [5]:
print(data.max())
print(data1.max())
print(data0.max())

0.855952701
4.5347185134887695
0.250600934028625


In [6]:

import numpy as np
data = np.array(data)
data1 = np.array(data1)
data0 = np.array(data0)
print(data.shape)
print(data1.shape)
print(data0.shape)

(546, 13)
(546, 13)
(546, 13)


In [7]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataKharif2017.csv', names=colnames)

In [8]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcan

In [9]:
p = np.array(ctype)
print(p.shape)

(547,)


In [10]:
lbl1 = ctype
label = []
for i in range(1,547):
    if(lbl1[i] == "Paddy"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "PearlMillet"):
        label += [2]
    if(lbl1[i] == "PigeonPea"):
        label += [3]
    if(lbl1[i] == "Sorghum"):
        label += [4]
    if(lbl1[i] == "Forest/NonCropland"):
        label += [5]
    if(lbl1[i] == "Habitation"):
        label += [6]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [11]:
label = np.array(label)
print(label.shape)

(546,)


In [12]:
print(data.shape)
print(data1.shape)
print(data0.shape)
data2 = np.concatenate((data,data1, data0), axis=1)
print(data2.shape)

(546, 13)
(546, 13)
(546, 13)
(546, 39)


In [20]:
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0,0,0,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0,0,0,0,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1,0,0,0,0])
    if(label[i]==3):
        x4.append(data2[i])
        l4.append([0,0,0,1,0,0,0])
    if(label[i]==4):
        x5.append(data2[i])
        l5.append([0,0,0,0,1,0,0])
    if(label[i]==5):
        x6.append(data2[i])
        l6.append([0,0,0,0,0,1,0])
    if(label[i]==6):
        x7.append(data2[i])
        l7.append([0,0,0,0,0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
x4=np.array(x4)
x5=np.array(x5)
x6=np.array(x6)
x7=np.array(x7)

l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)
l4=np.array(l4)
l5=np.array(l5)
l6=np.array(l6)
l7=np.array(l7)

print(l7.shape)
print(x6.shape)

(209, 7)
(34, 39)


In [13]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [14]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:436,:]
X_test = data2[436:,:]
y_train = label[0:436]
y_test = label[436:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(436, 39)
(110, 39)
(436,)
(110,)


In [15]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(436, 7)
(110, 7)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.07074169
Epoch: 2 cost = 0.94642917
Epoch: 3 cost = 0.83098030
Epoch: 4 cost = 0.72114531
Epoch: 5 cost = 0.62917767
Epoch: 6 cost = 0.55649193
Epoch: 7 cost = 0.49856522
Epoch: 8 cost = 0.45472006
Epoch: 9 cost = 0.42443397
Epoch: 10 cost = 0.40472235
Epoch: 11 cost = 0.39138593
Epoch: 12 cost = 0.38117488
Epoch: 13 cost = 0.37235123
Epoch: 14 cost = 0.36413527
Epoch: 15 cost = 0.35616978
Epoch: 16 cost = 0.34826591
Epoch: 17 cost = 0.34031656
Epoch: 18 cost = 0.33228679
Epoch: 19 cost = 0.32422686
Epoch: 20 cost = 0.31625705
Epoch: 21 cost = 0.30850520
Epoch: 22 cost = 0.30104008
Epoch: 23 cost = 0.29385245
Epoch: 24 cost = 0.28690640
Epoch: 25 cost = 0.28023340
Epoch: 26 cost = 0.27399455
Epoch: 27 cost = 0.26837324
Epoch: 28 cost = 0.26314006
Epoch: 29 cost = 0.25734312
Epoch: 30 cost = 0.25013329
Epoch: 31 cost = 0.24177512
Epoch: 32 cost = 0.23970696
Epoch: 33 cost = 0.39514925
Epoch: 34 cost = 0.50552700
Epoch: 35 cost = 0.20708065
Epoch: 36 cost = 0.18764973
E

Epoch: 309 cost = 0.01668257
Epoch: 310 cost = 0.01668596
Epoch: 311 cost = 0.01666103
Epoch: 312 cost = 0.01649273
Epoch: 313 cost = 0.01657704
Epoch: 314 cost = 0.01753299
Epoch: 315 cost = 0.01707684
Epoch: 316 cost = 0.01697005
Epoch: 317 cost = 0.01667697
Epoch: 318 cost = 0.01809058
Epoch: 319 cost = 0.27333407
Epoch: 320 cost = 0.60130167
Epoch: 321 cost = 0.47010165
Epoch: 322 cost = 0.12395097
Epoch: 323 cost = 0.12193577
Epoch: 324 cost = 0.02904219
Epoch: 325 cost = 0.02090363
Epoch: 326 cost = 0.01938135
Epoch: 327 cost = 0.01874405
Epoch: 328 cost = 0.01829081
Epoch: 329 cost = 0.01840471
Epoch: 330 cost = 0.01774988
Epoch: 331 cost = 0.01793336
Epoch: 332 cost = 0.01761552
Epoch: 333 cost = 0.01771226
Epoch: 334 cost = 0.01731083
Epoch: 335 cost = 0.01723154
Epoch: 336 cost = 0.01735162
Epoch: 337 cost = 0.01744048
Epoch: 338 cost = 0.01743236
Epoch: 339 cost = 0.01741120
Epoch: 340 cost = 0.01716176
Epoch: 341 cost = 0.01723461
Epoch: 342 cost = 0.01667101
Epoch: 343 cos

Epoch: 626 cost = 0.01125766
Epoch: 627 cost = 0.01085438
Epoch: 628 cost = 0.01082774
Epoch: 629 cost = 0.01076216
Epoch: 630 cost = 0.01095015
Epoch: 631 cost = 0.01115075
Epoch: 632 cost = 0.01145248
Epoch: 633 cost = 0.01109540
Epoch: 634 cost = 0.01088140
Epoch: 635 cost = 0.01095945
Epoch: 636 cost = 0.01106420
Epoch: 637 cost = 0.01094194
Epoch: 638 cost = 0.01090326
Epoch: 639 cost = 0.01101709
Epoch: 640 cost = 0.01096960
Epoch: 641 cost = 0.01097085
Epoch: 642 cost = 0.01109080
Epoch: 643 cost = 0.01090023
Epoch: 644 cost = 0.01136725
Epoch: 645 cost = 0.01570526
Epoch: 646 cost = 0.58828309
Epoch: 647 cost = 0.04620380
Epoch: 648 cost = 0.01205769
Epoch: 649 cost = 0.01168370
Epoch: 650 cost = 0.01121181
Epoch: 651 cost = 0.01110559
Epoch: 652 cost = 0.01106361
Epoch: 653 cost = 0.01100123
Epoch: 654 cost = 0.01097314
Epoch: 655 cost = 0.01134719
Epoch: 656 cost = 0.01111347
Epoch: 657 cost = 0.01099436
Epoch: 658 cost = 0.01107066
Epoch: 659 cost = 0.01129938
Epoch: 660 cos

Epoch: 939 cost = 0.01088367
Epoch: 940 cost = 0.01085034
Epoch: 941 cost = 0.01128167
Epoch: 942 cost = 0.10331192
Epoch: 943 cost = 0.08741811
Epoch: 944 cost = 0.01192003
Epoch: 945 cost = 0.01108301
Epoch: 946 cost = 0.01102892
Epoch: 947 cost = 0.01083803
Epoch: 948 cost = 0.01110847
Epoch: 949 cost = 0.01097636
Epoch: 950 cost = 0.01068396
Epoch: 951 cost = 0.01081527
Epoch: 952 cost = 0.01083128
Epoch: 953 cost = 0.01086845
Epoch: 954 cost = 0.01091413
Epoch: 955 cost = 0.01069115
Epoch: 956 cost = 0.01081713
Epoch: 957 cost = 0.01053798
Epoch: 958 cost = 0.01053066
Epoch: 959 cost = 0.01099826
Epoch: 960 cost = 0.01104556
Epoch: 961 cost = 0.01063117
Epoch: 962 cost = 0.01068275
Epoch: 963 cost = 0.01061048
Epoch: 964 cost = 0.01046490
Epoch: 965 cost = 0.01084363
Epoch: 966 cost = 0.01105062
Epoch: 967 cost = 0.01054017
Epoch: 968 cost = 0.01044734
Epoch: 969 cost = 0.01078491
Epoch: 970 cost = 0.01098854
Epoch: 971 cost = 0.01100609
Epoch: 972 cost = 0.01077012
Epoch: 973 cos

In [17]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
print(y_test.shape)

(110, 7)


In [21]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8272727


In [22]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
res4=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x4, Y: l4}))
res5=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x5, Y: l5}))
res6=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x6, Y: l6}))
res7=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x7, Y: l7}))
print(res6)

[5 5 5 5 5 5 1 5 5 5 1 5 5 5 5 5 1 5 5 1 5 5 5 5 5 5 1 5 5 5 5 5 5 1]


In [23]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
x4=np.sum(np.equal(res1, 3))
x5=np.sum(np.equal(res1, 4))
x6=np.sum(np.equal(res1, 5))
x7=np.sum(np.equal(res1, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
x4=np.sum(np.equal(res2, 3))
x5=np.sum(np.equal(res2, 4))
x6=np.sum(np.equal(res2, 5))
x7=np.sum(np.equal(res2, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
x4=np.sum(np.equal(res3, 3))
x5=np.sum(np.equal(res3, 4))
x6=np.sum(np.equal(res3, 5))
x7=np.sum(np.equal(res3, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res4, 0))
x2=np.sum(np.equal(res4, 1))
x3=np.sum(np.equal(res4, 2))
x4=np.sum(np.equal(res4, 3))
x5=np.sum(np.equal(res4, 4))
x6=np.sum(np.equal(res4, 5))
x7=np.sum(np.equal(res4, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res5, 0))
x2=np.sum(np.equal(res5, 1))
x3=np.sum(np.equal(res5, 2))
x4=np.sum(np.equal(res5, 3))
x5=np.sum(np.equal(res5, 4))
x6=np.sum(np.equal(res5, 5))
x7=np.sum(np.equal(res5, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res6, 0))
x2=np.sum(np.equal(res6, 1))
x3=np.sum(np.equal(res6, 2))
x4=np.sum(np.equal(res6, 3))
x5=np.sum(np.equal(res6, 4))
x6=np.sum(np.equal(res6, 5))
x7=np.sum(np.equal(res6, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res7, 0))
x2=np.sum(np.equal(res7, 1))
x3=np.sum(np.equal(res7, 2))
x4=np.sum(np.equal(res7, 3))
x5=np.sum(np.equal(res7, 4))
x6=np.sum(np.equal(res7, 5))
x7=np.sum(np.equal(res7, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

confusion=np.array(confusion)
print(confusion)

[[ 48   9   1   0   3   0   0]
 [  5 154   2   0   1  10   0]
 [ 14   5   8   0   6   0   0]
 [  1   8   0   0   1   0   0]
 [  4   6   4   0  12   0   1]
 [  0   6   0   0   0  28   0]
 [  2   6   0   0   1   4 196]]


In [24]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:382,:]
X_test = data2[382:,:]
y_train = label[0:382]
y_test = label[382:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(382, 39)
(164, 39)
(382,)
(164,)


In [25]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(382, 7)
(164, 7)


In [26]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.14582357
Epoch: 2 cost = 1.09918365
Epoch: 3 cost = 1.05795012
Epoch: 4 cost = 1.01922102
Epoch: 5 cost = 0.98074441
Epoch: 6 cost = 0.94114423
Epoch: 7 cost = 0.89966390
Epoch: 8 cost = 0.85580435
Epoch: 9 cost = 0.80897660
Epoch: 10 cost = 0.75815306
Epoch: 11 cost = 0.70209169
Epoch: 12 cost = 0.64195004
Epoch: 13 cost = 0.58583198
Epoch: 14 cost = 0.54410691
Epoch: 15 cost = 0.51582141
Epoch: 16 cost = 0.49246404
Epoch: 17 cost = 0.47029557
Epoch: 18 cost = 0.44917169
Epoch: 19 cost = 0.42950829
Epoch: 20 cost = 0.41115927
Epoch: 21 cost = 0.39338670
Epoch: 22 cost = 0.37551478
Epoch: 23 cost = 0.35799327
Epoch: 24 cost = 0.34236950
Epoch: 25 cost = 0.33028935
Epoch: 26 cost = 0.32431366
Epoch: 27 cost = 0.33155649
Epoch: 28 cost = 0.36819334
Epoch: 29 cost = 0.38487017
Epoch: 30 cost = 0.28390051
Epoch: 31 cost = 0.26981933
Epoch: 32 cost = 0.29374143
Epoch: 33 cost = 0.28663515
Epoch: 34 cost = 0.26248275
Epoch: 35 cost = 0.50849295
Epoch: 36 cost = 0.72871010
E

Epoch: 304 cost = 0.53156596
Epoch: 305 cost = 0.05386441
Epoch: 306 cost = 0.01450342
Epoch: 307 cost = 0.01133407
Epoch: 308 cost = 0.01006748
Epoch: 309 cost = 0.00930057
Epoch: 310 cost = 0.00852710
Epoch: 311 cost = 0.00845050
Epoch: 312 cost = 0.00780716
Epoch: 313 cost = 0.00779618
Epoch: 314 cost = 0.00737199
Epoch: 315 cost = 0.00771303
Epoch: 316 cost = 0.00771320
Epoch: 317 cost = 0.00706981
Epoch: 318 cost = 0.00745743
Epoch: 319 cost = 0.00751283
Epoch: 320 cost = 0.00741466
Epoch: 321 cost = 0.00682990
Epoch: 322 cost = 0.00736682
Epoch: 323 cost = 0.00660963
Epoch: 324 cost = 0.00594494
Epoch: 325 cost = 0.00647623
Epoch: 326 cost = 0.00642083
Epoch: 327 cost = 0.00675090
Epoch: 328 cost = 0.00638206
Epoch: 329 cost = 0.00589423
Epoch: 330 cost = 0.00584807
Epoch: 331 cost = 0.00599631
Epoch: 332 cost = 0.00600844
Epoch: 333 cost = 0.00748770
Epoch: 334 cost = 0.00683867
Epoch: 335 cost = 0.00631289
Epoch: 336 cost = 0.00601016
Epoch: 337 cost = 0.00667214
Epoch: 338 cos

Epoch: 626 cost = 0.00491335
Epoch: 627 cost = 0.00507806
Epoch: 628 cost = 0.00500896
Epoch: 629 cost = 0.00489656
Epoch: 630 cost = 0.00460487
Epoch: 631 cost = 0.00498254
Epoch: 632 cost = 0.00459626
Epoch: 633 cost = 0.00465719
Epoch: 634 cost = 0.00992512
Epoch: 635 cost = 0.87114618
Epoch: 636 cost = 0.11737670
Epoch: 637 cost = 0.00892157
Epoch: 638 cost = 0.00701681
Epoch: 639 cost = 0.00584675
Epoch: 640 cost = 0.00554493
Epoch: 641 cost = 0.00597293
Epoch: 642 cost = 0.00507439
Epoch: 643 cost = 0.00490961
Epoch: 644 cost = 0.00491097
Epoch: 645 cost = 0.00491759
Epoch: 646 cost = 0.00502890
Epoch: 647 cost = 0.00543259
Epoch: 648 cost = 0.00475799
Epoch: 649 cost = 0.00481411
Epoch: 650 cost = 0.00504900
Epoch: 651 cost = 0.00537013
Epoch: 652 cost = 0.00450065
Epoch: 653 cost = 0.00436820
Epoch: 654 cost = 0.00477467
Epoch: 655 cost = 0.00535436
Epoch: 656 cost = 0.00494155
Epoch: 657 cost = 0.00475327
Epoch: 658 cost = 0.00469342
Epoch: 659 cost = 0.00449056
Epoch: 660 cos

Epoch: 943 cost = 0.00399040
Epoch: 944 cost = 0.00453673
Epoch: 945 cost = 0.00488353
Epoch: 946 cost = 0.00425664
Epoch: 947 cost = 0.00392200
Epoch: 948 cost = 0.00421464
Epoch: 949 cost = 0.00471742
Epoch: 950 cost = 0.00461855
Epoch: 951 cost = 0.00460493
Epoch: 952 cost = 0.00397084
Epoch: 953 cost = 0.00452918
Epoch: 954 cost = 0.00485851
Epoch: 955 cost = 0.00440949
Epoch: 956 cost = 0.00434046
Epoch: 957 cost = 0.00507729
Epoch: 958 cost = 0.00588796
Epoch: 959 cost = 0.41547924
Epoch: 960 cost = 0.01498339
Epoch: 961 cost = 0.07645036
Epoch: 962 cost = 0.00523566
Epoch: 963 cost = 0.00462678
Epoch: 964 cost = 0.00479520
Epoch: 965 cost = 0.00485831
Epoch: 966 cost = 0.00460136
Epoch: 967 cost = 0.00426356
Epoch: 968 cost = 0.00446185
Epoch: 969 cost = 0.00447110
Epoch: 970 cost = 0.00433336
Epoch: 971 cost = 0.00449397
Epoch: 972 cost = 0.00451306
Epoch: 973 cost = 0.00399211
Epoch: 974 cost = 0.00421470
Epoch: 975 cost = 0.00430244
Epoch: 976 cost = 0.00423093
Epoch: 977 cos

In [27]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.81097555
